# Setup enviorment

In [1]:
# Data reading in Dataframe format and data preprocessing
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Linear algebra operations
import numpy as np

# Image processing
from skimage import io
from skimage.transform import resize

# Machine learning models and preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Flatten
import tensorflow.keras.backend as K

# Epiweek
from epiweeks import Week, Year

# Date
from datetime import date as convert_to_date

# Os
import os

# Feature Extraction Model:
from Autoencoder_Architecture import get_Autoencoder

tf.compat.v1.experimental.output_all_intermediates(True)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
features = 'DATASET_5_best_cities/'
labels = 'Tabular_data/Label_CSV_All_Municipality.csv'
MUNICIPALITY = 'Ibagué'

target_size = (224, 224, 12)
backbone = 'Models/autoencoder_224_1024.h5'

cities =  {
  "76001": "Cali",
  "05001": "Medellín",
  "50001": "Villavicencio",
  "54001": "Cúcuta",
  "73001": "Ibagué",
  "68001": "Bucaramanga",
  "05360": "Itagüí",
  "08001": "Barranquilla",
  "41001": "Neiva",
  "23001": "Montería"
}

codes =  {
  "Cali": "76001",
  "Medellín": "05001",
  "Villavicencio": "50001",
  "Cúcuta": "54001",
  "Ibagué": "73001",
  "Bucaramanga": "68001",
  "Itagüí": "05360",
  "Barranquilla": "08001",
  "Neiva": "41001",
  "Montería": "23001"
}

# Read Data

In [4]:
def epiweek_from_date(image_date):
    date = image_date.split('-')
    
    # Get year as int
    year = ''.join(filter(str.isdigit, date[0]))
    year = int(year)
    
    # Get month as int
    month = ''.join(filter(str.isdigit, date[1]))
    month = int(month)
    
    # Get day as int
    day = ''.join(filter(str.isdigit, date[2]))
    day = int(day)
    
    # Get epiweek:
    date = convert_to_date(year, month, day)
    epiweek = str(Week.fromdate(date))
    epiweek = int(epiweek)
    
    return epiweek

In [5]:
def get_epiweek(name):
    
    # Get week
    week = name.split('/')[1]
    week = week.replace('w','')
    week = int(week)
    
    # Year
    year = name.split('/')[0]
    year = int(year)
    
    epiweek = Week(year, week)
    
    epiweek = str(epiweek)
    epiweek = int(epiweek)

    return epiweek

In [6]:
def read_labels(path, Municipality = None):
    df = pd.read_csv(path)
    if df.shape[1] > 678:
        df = pd.concat([df[['Municipality code', 'Municipality']], df.iloc[:,-676:]], axis=1)
        cols = df.iloc[:, 2:].columns
        new_cols = df.iloc[:, 2:].columns.to_series().apply(get_epiweek)
        df = df.rename(columns=dict(zip(cols, new_cols))) 
        
    if 'Label_CSV_All_Municipality' in path:
        # Get Columns
        df = df[['epiweek', 'Municipality code', 'Municipality', 'final_cases_label']]
        
        # change epiweek format
        df.epiweek = df.epiweek.apply(get_epiweek)
        
        # Remove duplicates
        df = df[df.duplicated(['epiweek','Municipality code','Municipality']) == False]
        
        # Replace Increase, decrease, stable to numerical:
        """
        - Stable = 0
        - Increased = 1 
        - Decreased = 2
        """
        df.final_cases_label = df.final_cases_label.replace({'Stable': 0, 'Increased': 1, 'Decreased': 2})
        
        # Create table
        df = df.pivot(index=['Municipality code', 'Municipality'], columns='epiweek', values='final_cases_label')

        # Reset Index:
        df = df.reset_index()
    
    if Municipality:
        df = df[df['Municipality'] == Municipality]
        df.drop(columns=['Municipality code'], inplace=True)
        df.rename(columns={'Municipality': 'Municipality Code'}, inplace=True)
    
        df = df.set_index('Municipality Code')
        df = df.T

        df.columns.name = None
        df.index.name = None
        
        df.columns = ['Cases']
    
    #df = df.reset_index()
    #df.rename(columns={'index': 'epiweek'}, inplace=True)
    return df

### Labels

In [7]:
labels_df = read_labels(path=labels, Municipality=MUNICIPALITY)
labels_df_orig = labels_df
labels_df = pd.get_dummies(labels_df['Cases'])
labels_df

,0,1,2
201601,1,0,0
201602,1,0,0
201603,0,1,0
201604,0,0,1
201605,0,1,0
...,...,...,...
201848,0,1,0
201849,0,0,1
201850,1,0,0
201851,1,0,0


### Features

In [8]:
def create_df(images_dir):
    
    out_df = {
        'epiweek':[],
        'image':[]
    }
    
    for image_path in os.listdir(images_dir):
        if image_path.endswith('.tiff'):
            epiweek = epiweek_from_date(image_path)
            full_path = os.path.join(images_dir, image_path)
            
            out_df['epiweek'].append(epiweek)
            out_df['image'].append(full_path)

    df = pd.DataFrame(out_df)
    
    df = df.set_index('epiweek')
    df.index.name = None
    
    return df

In [9]:
if 'DATASET_5_best_cities' in features:
    MUNICIPALITY = MUNICIPALITY
else:
    MUNICIPALITY = codes[MUNICIPALITY]
    
images_dir = os.path.join(features, MUNICIPALITY)

features_df = create_df(images_dir)
features_df.head()

,image
201731,DATASET_5_best_cities/Ibagué/image_2017-07-30....
201551,DATASET_5_best_cities/Ibagué/image_2015-12-20....
201747,DATASET_5_best_cities/Ibagué/image_2017-11-19....
201647,DATASET_5_best_cities/Ibagué/image_2016-11-20....
201729,DATASET_5_best_cities/Ibagué/image_2017-07-16....


In [10]:
def read_image(path, target_size):
    # Read the image and convert to numpy array
    image = io.imread(path)
    # Resize the image and normalize values
    image_arr = resize(image,(target_size[0], target_size[1]))
    #print(f'The shape of the image before reshape: {image_arr.shape}, of type{type(image_arr)}')

    # Select RGB bands
    if target_size[2] == 3:
        image_arr = image_arr[:,:, [1,2,3]]
    return image_arr

In [11]:
features_df.image = features_df.image.apply(read_image, args=[target_size])
#features_df.head()

# Data preparation

In [12]:
n_labels = labels_df.shape[1]

In [13]:
# Merge the two dataframes based on the date values
dengue_df = features_df.merge(labels_df, how='inner', left_index=True, right_index=True)
dengue_df = dengue_df.sort_index()
#dengue_df.head()

### Train Test split

In [14]:
def train_test_split(df, train_percentage = 80):
    # We need a sequence so we can't split randomly
    # To divide into Train and test we have to calculate the train percentage of the dataset:
    size = df.shape[0]
    split = int(size*(train_percentage/100))
    
    """ Train """
    # We will train with 1st percentage % of data and test with the rest
    train_df = df.iloc[:split,:] ## percentage % train
    
    """ Test """
    test_df = df.iloc[split:,:] # 100 - percentage % test
    
    print(f'The train shape is: {train_df.shape}')
    print(f'The test shape is: {test_df.shape}')
    
    return train_df, test_df

In [15]:
train_df, test_df = train_test_split(dengue_df, train_percentage = 80)

The train shape is: (124, 4)
The test shape is: (32, 4)


### Normalization

In [16]:
def normalize_train_labels(df, column, feature_range=(-1, 1)):
    # Get values of the column
    values = df[column].values.reshape(-1,1)
    # Generate a new scaler
    scaler = MinMaxScaler(feature_range=feature_range)
    # Fit the scaler just for that column
    scaled_column = scaler.fit_transform(values)
    # Add the scaled column to the dataframe
    scaled_column = np.reshape(scaled_column, len(scaled_column))
    df[column] = scaled_column
    return df, scaler
    
def normalize_test_labels(df, column, scaler):
    # Get values of the column
    values = df[column].values.reshape(-1,1)
    # Scale values
    scaled_column = scaler.transform(values)
    scaled_column = np.reshape(scaled_column,len(scaled_column))
    # Add the scaled values to the df
    df[column] = scaled_column
    return df
    

In [17]:
feature_range = (-1, 1)

# Scale train:
#train_df, scaler = normalize_train_labels(train_df, 'Cases', feature_range=feature_range)

#train_df['Cases'].head()

# Scale test:
#test_df = normalize_test_labels(test_df, 'Cases', scaler=scaler)
#test_df['Cases'].head()

### Prepare data for time series supervised learning (function to create sliding window)

In [18]:
# prepare data for time series

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, no_autoregressive=None):
    if no_autoregressive:
        n_in = n_in - 1
        
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        if no_autoregressive:
            cols.append(df.shift(i).iloc[:,:-n_labels])
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars-n_labels)]
        else:
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [19]:
# length of window
days = 10
no_autoregressive = True

# frame as supervised learning
train = series_to_supervised(train_df, n_in=days, no_autoregressive=no_autoregressive)
test = series_to_supervised(test_df, n_in=days, no_autoregressive=no_autoregressive)

#DataFrame(train)

In [20]:
def convert_df_to_np(train):
    for i, column in enumerate(train.columns):
        if i == 0:
            train_arr = np.array(train[column].to_list())
            train_arr = np.expand_dims(train_arr, axis=1)

        else:
            #print(f'original: {train_arr.shape}')

            train_arr_aux = np.array(train[column].to_list())
            train_arr_aux = np.expand_dims(train_arr_aux, axis=1)

            #print(f'aux: {train_arr_aux.shape}')

            train_arr = np.concatenate((train_arr, train_arr_aux), axis=1)

    return train_arr

### Features and Labels Set

In [21]:
def features_labels_set(timeseries_data, original_df):
    
    """ Features """
    # We define the number of features as (Cases and media cloud)
    n_features = original_df.shape[1]

    # The features to train the model will be all except the values of the actual week 
    # We can't use other variables in week t because whe need to resample a a 3D Array
    features_set = DataFrame(timeseries_data.values[:,:-n_labels])
    # Convert pandas data frame to np.array to reshape as 3D Array
    features_set = convert_df_to_np(features_set)
    print(f'The shape of the features is {features_set.shape}')
    
    """ Labels """
    # We will use Covid cases in last week 
    labels_set = DataFrame(timeseries_data.values[:,-n_labels:])
    # Convert pandas data frame to np.array
    labels_set = labels_set.to_numpy()
    print(f'The shape of the labels is {labels_set.shape}')
    
    return features_set, labels_set, n_features

In [22]:
# Train features and labels set
print('Train:')
train_X, train_y, n_features = features_labels_set(timeseries_data=train, original_df=dengue_df)

# Test features and labels set
print('Test:')
test_X, test_y, n_features = features_labels_set(timeseries_data=test, original_df=dengue_df)

Train:
The shape of the features is (115, 10, 224, 224, 12)
The shape of the labels is (115, 3)
Test:
The shape of the features is (23, 10, 224, 224, 12)
The shape of the labels is (23, 3)


In [23]:
train_X = np.asarray(train_X).astype(np.float32)
train_y = np.asarray(train_y).astype(np.float32)

test_X = np.asarray(test_X).astype(np.float32)
test_y = np.asarray(test_y).astype(np.float32)

# Define the Model

In [24]:
# Set Seed
#tf.random.set_seed(0)

def smape(y_true, y_pred):
    epsilon = 0.1
    summ = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = K.abs(y_pred - y_true) / summ * 2.0
    return smape

def create_model(backbone=backbone):
    lstm_week, input_shape = days, target_size
    
    # design network
    model = Sequential()

    # CNN
    cnn = get_Autoencoder(model_path=backbone, backbone=True)

    for idx, layer in enumerate(cnn.layers):
        layer.trainable = False # idx > len(cnn.layers) - 2 
    
    # https://levelup.gitconnected.com/hands-on-practice-with-time-distributed-layers-using-tensorflow-c776a5d78e7e
    model.add(keras.layers.TimeDistributed(cnn, input_shape = ((lstm_week,) + input_shape)))
    model.add(keras.layers.TimeDistributed(Flatten()))
    model.add(keras.layers.TimeDistributed(Dense(1024)))
    model.add(LSTM(120, dropout=0.1, return_sequences=True))
    model.add(LSTM(240, dropout=0.1, return_sequences = False))
    model.add(Dense(60))
    model.add(Dense(3, activation='softmax'))
    #print(model.summary())
    
    # Compile the model:
    opt = keras.optimizers.Adam()
    metrics = [
        tf.keras.metrics.AUC(name='auc', multi_label=True, num_labels=3),
        tf.keras.metrics.CategoricalAccuracy(name='acc'),
        tf.keras.metrics.CategoricalCrossentropy(name='entropy')
    ]

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=metrics)
    

    return model

### Train the model

In [25]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, 
        verbose=1, mode='auto', restore_best_weights=True)

In [26]:
### Imbalanced data
n_zeros = (labels_df_orig.to_numpy() == 0).sum()
n_ones = (labels_df_orig.to_numpy() == 1).sum()
n_twos = (labels_df_orig.to_numpy() == 2).sum()
n_total = n_zeros + n_ones + n_twos

weights = {0: n_total/n_zeros, 1: n_total/n_ones, 2: n_total/n_twos}
print(f'zeros: {n_zeros}, ones: {n_ones}, twos: {n_twos}, total: {n_total}')
weights

zeros: 104, ones: 23, twos: 29, total: 156


{0: 1.5, 1: 6.782608695652174, 2: 5.379310344827586}

In [27]:
# fit network
def train_model(model, monitor, weights, plot=None, epochs=20):
    if monitor and weights:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[monitor], class_weight=weights)
    elif monitor:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[monitor])
    elif weights:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, class_weight=weights)
    else:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    
    if plot:
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.legend()
        plt.show()

# AUC

In [28]:
# You can also evaluate or predict on a dataset.
def evaluate(model, verbose = None):
    if verbose:
        print('Evaluate: ')
    result = model.evaluate(test_X, test_y)
    stored_results = {}
    for i, metric in enumerate(model.metrics_names):
        stored_results[metric] = result[i]
        if verbose:
            print(f'{metric}: {result[i]}')
    return stored_results

In [29]:
def plot_predictions(inv_y, inv_yhat, model_name = ''):
    data_predict = inv_yhat  ## predicted target cases
    dataY_plot = inv_y  ##  real test-target cases

    data_predict = data_predict.reshape(len(data_predict), 1)
    dataY_plot = dataY_plot.reshape(len(dataY_plot), 1)

    plt.plot(dataY_plot, label = 'actual')
    plt.plot(data_predict, label = 'predicted')
    plt.legend(loc="upper left")
    plt.suptitle(f'Time-Series Prediction with {model_name}')
    plt.show()

# Calculate Mean and SD

In [30]:
def calculate_mean_std(weights):
    
    metrics = {
        "auc": [],
        "acc": [],
        "entropy": []
    }
    
    for i in range(3):
        model = create_model(backbone=backbone)
        train_model(model=model, monitor=monitor, weights=weights)
        stored_results = evaluate(model=model)
        
        for key in metrics.keys():
            metrics[key].append(stored_results[key])
            
    for key in metrics.keys():
        results = metrics[key]
        print(key, f": average={np.average(results):.3f}, std={np.std(results):.3f}")

In [31]:
calculate_mean_std(weights=None)
#calculate_mean_std(weights=weights)

Instructions for updating:
Colocations handled automatically by placer.
Train on 115 samples, validate on 23 samples
Epoch 1/20
115/115 - 7s - loss: 0.8584 - auc: 0.6627 - acc: 0.6348 - entropy: 0.8584 - val_loss: 0.7011 - val_auc: 0.4807 - val_acc: 0.8261 - val_entropy: 0.7011
Epoch 2/20
115/115 - 5s - loss: 1.1935 - auc: 0.3024 - acc: 0.6609 - entropy: 1.1935 - val_loss: 0.6106 - val_auc: 0.5479 - val_acc: 0.8261 - val_entropy: 0.6106
Epoch 3/20
115/115 - 5s - loss: 0.8899 - auc: 0.4355 - acc: 0.6609 - entropy: 0.8899 - val_loss: 0.5945 - val_auc: 0.6751 - val_acc: 0.8261 - val_entropy: 0.5945
Epoch 4/20
115/115 - 4s - loss: 0.8828 - auc: 0.5073 - acc: 0.6609 - entropy: 0.8828 - val_loss: 0.5979 - val_auc: 0.6201 - val_acc: 0.8261 - val_entropy: 0.5979
Epoch 5/20
115/115 - 5s - loss: 0.8727 - auc: 0.5347 - acc: 0.6609 - entropy: 0.8727 - val_loss: 0.5829 - val_auc: 0.6443 - val_acc: 0.8261 - val_entropy: 0.5829
Epoch 6/20
115/115 - 5s - loss: 0.8667 - auc: 0.5786 - acc: 0.6609 - entr